UNLABELED DATASET - UNSUPERVISED LEARNING

The bank statement dataset does not contain ground-truth fraud labels. Therefore, supervised fraud classification is not directly possible on this data. We instead apply unsupervised anomaly detection and pattern discovery, and use LLMs to provide qualitative explanations of suspicious behaviour. For quantitative evaluation using classification metrics, we complement this with labeled open-source datasets.

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [42]:
df = pd.read_excel(r"C:\Users\louis\Downloads\R&D PROJECTS\data\bank.xlsx")

In [43]:
df.head()

,Account No,DATE,TRANSACTION DETAILS,CHQ.NO.,VALUE DATE,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,.
0,409000611074',2017-06-29,TRF FROM Indiaforensic SERVICES,NaN,2017-06-29,NaN,1000000.0,1000000.0,.
1,409000611074',2017-07-05,TRF FROM Indiaforensic SERVICES,NaN,2017-07-05,NaN,1000000.0,2000000.0,.
2,409000611074',2017-07-18,FDRL/INTERNAL FUND TRANSFE,NaN,2017-07-18,NaN,500000.0,2500000.0,.
3,409000611074',2017-08-01,TRF FRM Indiaforensic SERVICES,NaN,2017-08-01,NaN,3000000.0,5500000.0,.
4,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,NaN,2017-08-16,NaN,500000.0,6000000.0,.


In [44]:
#Clean up the column names
df.columns = [
    "account_no",
    "date",
    "transaction_details",
    "cheque_no",
    "value_date",
    "withdrawal_amt",
    "deposit_amt",
    "balance_amt",
    "drop_col"
]

df.head()

,account_no,date,transaction_details,cheque_no,value_date,withdrawal_amt,deposit_amt,balance_amt,drop_col
0,409000611074',2017-06-29,TRF FROM Indiaforensic SERVICES,NaN,2017-06-29,NaN,1000000.0,1000000.0,.
1,409000611074',2017-07-05,TRF FROM Indiaforensic SERVICES,NaN,2017-07-05,NaN,1000000.0,2000000.0,.
2,409000611074',2017-07-18,FDRL/INTERNAL FUND TRANSFE,NaN,2017-07-18,NaN,500000.0,2500000.0,.
3,409000611074',2017-08-01,TRF FRM Indiaforensic SERVICES,NaN,2017-08-01,NaN,3000000.0,5500000.0,.
4,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,NaN,2017-08-16,NaN,500000.0,6000000.0,.


In [45]:
#Change the date value from string to date_time
df["date"] = pd.to_datetime(df["date"])
df["value_date"] = pd.to_datetime(df["value_date"])
df.head()

,account_no,date,transaction_details,cheque_no,value_date,withdrawal_amt,deposit_amt,balance_amt,drop_col
0,409000611074',2017-06-29,TRF FROM Indiaforensic SERVICES,NaN,2017-06-29,NaN,1000000.0,1000000.0,.
1,409000611074',2017-07-05,TRF FROM Indiaforensic SERVICES,NaN,2017-07-05,NaN,1000000.0,2000000.0,.
2,409000611074',2017-07-18,FDRL/INTERNAL FUND TRANSFE,NaN,2017-07-18,NaN,500000.0,2500000.0,.
3,409000611074',2017-08-01,TRF FRM Indiaforensic SERVICES,NaN,2017-08-01,NaN,3000000.0,5500000.0,.
4,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,NaN,2017-08-16,NaN,500000.0,6000000.0,.


In [46]:
#Clean the narration text (transaction details are messy and inconsistent)
df["transaction_clean"] = (
    df["transaction_details"]
    .astype(str)
    .str.lower()
    .str.replace(r"[^a-z0-9\s]", "", regex=True)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

# Fix NaN text
df["transaction_clean"] = df["transaction_clean"].replace("nan", "unknown_transaction")
df["transaction_clean"] = df["transaction_clean"].fillna("unknown_transaction")

df[["transaction_details", "transaction_clean"]].head()

,transaction_details,transaction_clean
0,TRF FROM Indiaforensic SERVICES,trf from indiaforensic services
1,TRF FROM Indiaforensic SERVICES,trf from indiaforensic services
2,FDRL/INTERNAL FUND TRANSFE,fdrlinternal fund transfe
3,TRF FRM Indiaforensic SERVICES,trf frm indiaforensic services
4,FDRL/INTERNAL FUND TRANSFE,fdrlinternal fund transfe


In [47]:
#Create a unified amount column from deposit and withdrawal
#Positive = deposit , Negative = withdrawal
df["withdrawal_amt"] = df["withdrawal_amt"].fillna(0)
df["deposit_amt"] = df["deposit_amt"].fillna(0)

df["amount"] = df["deposit_amt"] - df["withdrawal_amt"]
df[["deposit_amt", "withdrawal_amt", "amount"]].head()

,deposit_amt,withdrawal_amt,amount
0,1000000.0,0.0,1000000.0
1,1000000.0,0.0,1000000.0
2,500000.0,0.0,500000.0
3,3000000.0,0.0,3000000.0
4,500000.0,0.0,500000.0


In [49]:
df["transaction_clean"] = df["transaction_clean"].replace("nan", "unknown_transaction")
df["transaction_clean"] = df["transaction_clean"].fillna("unknown_transaction")

In [50]:
df["transaction_clean"] = df["transaction_clean"].fillna("unknown_transaction")

In [51]:
#Build transaction sequences (LLM-ready text)
df_sorted = df.sort_values(["account_no", "date"])

sequence_df = (
    df_sorted.groupby("account_no")
    .apply(lambda x: " | ".join(
        "on " + x["date"].dt.strftime("%Y-%m-%d") +
        ", transaction: " + x["transaction_clean"] +
        ", amount: " + x["amount"].astype(str)
    ))
    .reset_index(name="transaction_sequence_text")
)

sequence_df.head()

,account_no,transaction_sequence_text
0,1196428',"on 2015-01-01, transaction: dsb cash pickp ind..."
1,1196711',"on 2015-01-01, transaction: 436315014201 rever..."
2,409000362497',"on 2015-07-16, transaction: unknown_transactio..."
3,409000405747',"on 2016-01-30, transaction: trf to maur joshi,..."
4,409000425051',"on 2016-02-23, transaction: neftsbin3160545194..."


In [21]:
#Basic ML feature extraction

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=1000, stop_words="english")
X_text = vectorizer.fit_transform(sequence_df["transaction_sequence_text"])

X_text.shape

(10, 1000)

In [52]:
#Pattern discovery (TF-IDF + Clustering)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

vectorizer = TfidfVectorizer(max_features=1000, stop_words="english")
X_text = vectorizer.fit_transform(sequence_df["transaction_sequence_text"])

kmeans = KMeans(n_clusters=5, random_state=42)
sequence_df["cluster"] = kmeans.fit_predict(X_text)

sequence_df[["account_no", "cluster"]].head()

C:\Users\louis\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


,account_no,cluster
0,1196428',1
1,1196711',1
2,409000362497',1
3,409000405747',1
4,409000425051',1


In [53]:
sequence_df["cluster"].value_counts()

1    6
0    1
4    1
3    1
2    1
Name: cluster, dtype: int64

In [64]:
#Match Same-Day Debit/Credit Across Accounts
df["abs_amount"] = df["amount"].abs()

matches = df.merge(
    df,
    on=["date", "abs_amount"],
    suffixes=("_a", "_b")
)

matches = matches[
    (matches["account_no_a"] != matches["account_no_b"]) &
    (matches["amount_a"] == -matches["amount_b"])
]

matches[[
    "date",
    "account_no_a", "amount_a", "transaction_details_a",
    "account_no_b", "amount_b", "transaction_details_b"
]].head()

,date,account_no_a,amount_a,transaction_details_a,account_no_b,amount_b,transaction_details_b
1,2017-06-29,409000611074',1000000.0,TRF FROM Indiaforensic SERVICES,1196428',-1000000.0,FDRL/INTERNAL FUND TRANSFE
3,2017-06-29,409000611074',1000000.0,TRF FROM Indiaforensic SERVICES,1196428',-1000000.0,TRF TO Indiaforensic SERVICES I
4,2017-06-29,1196428',-1000000.0,FDRL/INTERNAL FUND TRANSFE,409000611074',1000000.0,TRF FROM Indiaforensic SERVICES
12,2017-06-29,1196428',-1000000.0,TRF TO Indiaforensic SERVICES I,409000611074',1000000.0,TRF FROM Indiaforensic SERVICES
17,2017-07-05,409000611074',1000000.0,TRF FROM Indiaforensic SERVICES,1196428',-1000000.0,TRF TO Indiaforensic SERVICES I


Example (first row):

On 2017-06-29, account 409000611074 received 1,000,000 from Indiaforensic SERVICES, while account 1196428 sent 1,000,000 via internal fund transfer.

That’s a linked transaction pair. You’ve reconstructed the movement of money between accounts.

This is super relevant for:

Tracking fund flows

Detecting layering/circular movement

Supporting fraud investigations

**What we have so far: 
* Cleaned messy real-world financial text

* Built sequence-to-text representations

* Converted text into ML features

* Detected inter-account fund movements

Trained an AI model to automatically flag anomalous transactions as potential fraud candidates.

In [59]:
#AI Fraud/Anomaly Detection (Isolation Forest)
from sklearn.ensemble import IsolationForest

features = df[["amount"]].copy()

iso = IsolationForest(contamination=0.02, random_state=42)
df["anomaly_flag"] = iso.fit_predict(features)
df["is_suspicious"] = df["anomaly_flag"] == -1

df[["account_no", "date", "transaction_details", "amount", "is_suspicious"]].head(10)

,account_no,date,transaction_details,amount,is_suspicious
0,409000611074',2017-06-29,TRF FROM Indiaforensic SERVICES,1000000.0,False
1,409000611074',2017-07-05,TRF FROM Indiaforensic SERVICES,1000000.0,False
2,409000611074',2017-07-18,FDRL/INTERNAL FUND TRANSFE,500000.0,False
3,409000611074',2017-08-01,TRF FRM Indiaforensic SERVICES,3000000.0,False
4,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,500000.0,False
5,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,500000.0,False
6,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,500000.0,False
7,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,500000.0,False
8,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,500000.0,False
9,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,500000.0,False


**TESTING**

In [69]:
df["is_suspicious"].value_counts()

False    113877
True       2324
Name: is_suspicious, dtype: int64

In [70]:
print("matches shape:", matches.shape)

matches shape: (148458, 26)


In [62]:
df["is_suspicious"].value_counts(normalize=True)

False    0.98
True     0.02
Name: is_suspicious, dtype: float64

In [63]:
df[df["is_suspicious"]].head(5)[["account_no", "date", "transaction_details", "amount"]]

,account_no,date,transaction_details,amount
2926,409000425051',2018-10-31,TRF TO Myur Joshi,-354000000.0
2939,409000405747',2016-01-30,TRF TO Maur Joshi,-170000000.0
2940,409000405747',2016-01-30,TRF TO Indiaforensic SERVICES IN,-30000000.0
2950,409000405747',2016-05-03,TRF FROM Indiaforensic SERVICES,202100000.0
2972,409000405747',2018-09-28,Payments For : 9090000577,-103020294.2


✅ Cleaned real bank data

✅ Transaction sequences → text (LLM-ready)

✅ Pattern discovery (clusters)

✅ Fund-flow matching across accounts

✅ AI anomaly detection (fraud flags)

In [72]:
#Top risky accounts (by % suspicious transactions):
account_risk = (
    df.groupby("account_no")["is_suspicious"]
    .mean()
    .reset_index(name="suspicious_ratio")
    .sort_values("suspicious_ratio", ascending=False)
)


account_risk.head(10)

,account_no,suspicious_ratio
3,409000405747',0.078431
2,409000362497',0.047855
6,409000438620',0.027650
1,1196711',0.010251
0,1196428',0.008303
5,409000438611',0.001308
4,409000425051',0.001247
7,409000493201',0.000000
8,409000493210',0.000000
9,409000611074',0.000000


In [73]:
#Summarise fund flows (which account pairs move money the most)
pair_summary = (
    matches.groupby(["account_no_a", "account_no_b"])
    .agg(
        transfer_count=("amount_a", "count"),
        total_amount=("amount_a", "sum")
    )
    .reset_index()
    .sort_values( "transfer_count", ascending=False)
)

pair_summary.head(10)

,account_no_a,account_no_b,transfer_count,total_amount
1,1196428',409000362497',32741,-3.643221e+11
18,409000362497',1196428',32741,3.643221e+11
19,409000362497',1196711',28900,3.852453e+11
10,1196711',409000362497',28900,-3.852453e+11
0,1196428',1196711',3312,-1.467381e+10
9,1196711',1196428',3312,1.467381e+10
38,409000438611',1196428',2565,-4.458620e+08
4,1196428',409000438611',2565,4.458620e+08
5,1196428',409000438620',2070,9.061621e+09
47,409000438620',1196428',2070,-9.061621e+09


Evaluation (Unlabeled Data)
The dataset does not contain ground-truth fraud labels. Therefore, evaluation focuses on qualitative inspection of anomaly detection results and reconstructed fund flows. The anomaly detection model flags approximately 2% of transactions as suspicious, predominantly corresponding to large-value or infrequent transfers. Fund-flow matching successfully reconstructs same-day debit-credit movements across accounts, enabling analysis of inter-account fund movement patterns relevant to AML and fraud investigation.

The system surfaces these patterns to support human analysts, who would then conduct further investigation using additional contextual and regulatory information.

At the moment, the anomaly detection is based mainly on transaction size, so it flags unusually large or rare transfers as suspicious. It’s a baseline version, and we can extend it with behavioural features later.

In [ ]:
# LLM ?